In [2]:
# pip install python-binance
import pandas as pd
from binance.client import Client
client = Client()

In [5]:
def getData(symbol, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame.set_index('Time', inplace=True)
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

In [6]:
df = getData("BTCUSDT", "2023-01-01")

In [7]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16538.45,16543.67,83.08143
2023-01-01 00:01:00,16543.04,16544.41,16538.48,16539.31,80.45300
2023-01-01 00:02:00,16539.31,16541.17,16534.52,16536.43,62.90197
2023-01-01 00:03:00,16536.43,16537.28,16531.00,16533.65,115.71894
2023-01-01 00:04:00,16534.12,16536.08,16527.51,16535.38,144.45369
...,...,...,...,...,...
2023-09-16 10:58:00,26531.43,26535.14,26531.43,26535.13,8.78796
2023-09-16 10:59:00,26535.13,26545.19,26535.13,26545.18,8.04274
2023-09-16 11:00:00,26545.18,26545.19,26545.18,26545.19,6.24110


In [12]:
opens = df.resample("D").first().Open
opens

Time
2023-01-01    16541.77
2023-01-02    16617.17
2023-01-03    16672.78
2023-01-04    16675.65
2023-01-05    16850.36
                ...   
2023-09-12    25162.53
2023-09-13    25840.10
2023-09-14    26222.00
2023-09-15    26522.73
2023-09-16    26599.99
Freq: D, Name: Open, Length: 259, dtype: float64

In [10]:
def getlevels(date,first=True):
    if first:
        return opens[date] * 0.998, opens[date] * 1.002
    else:
        return opens[date] * 0.996, opens[date] * 0.998

In [39]:
getlevels('2023-01-01')

(16508.68646, 16574.85354)

In [64]:
import datetime

def slicedf(date):
    return df[df.index.date == pd.to_datetime(date)]
    # date_obj = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    # return df[df.index.date == date_obj]

In [43]:
df_t = slicedf("2023-01-01")
df_t

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-01 00:00:00,16541.77,16544.76,16538.45,16543.67,83.08143
2023-01-01 00:01:00,16543.04,16544.41,16538.48,16539.31,80.45300
2023-01-01 00:02:00,16539.31,16541.17,16534.52,16536.43,62.90197
2023-01-01 00:03:00,16536.43,16537.28,16531.00,16533.65,115.71894
2023-01-01 00:04:00,16534.12,16536.08,16527.51,16535.38,144.45369
...,...,...,...,...,...
2023-01-01 23:55:00,16609.48,16615.09,16609.19,16613.60,70.85806
2023-01-01 23:56:00,16613.20,16621.00,16612.15,16619.79,91.04320
2023-01-01 23:57:00,16620.20,16621.44,16617.19,16617.20,74.52966


In [44]:
in_position = False

for index,row in df_t.iterrows():
    if not in_position:
        levels = getlevels('2023-01-01')
        if row.Low <= levels[0]:
            print(index)
            print('buy')
            in_position = True
    if in_position:
        if row.High >= levels[1]:
            print(index)
            print('sold')
            in_position = False

2023-01-01 00:37:00
buy
2023-01-01 16:02:00
sold


In [56]:
# desired_datetime = datetime.datetime('2023-01-01 16:00:00')
# data_at_desired_datetime = df_t.loc[desired_datetime]

desired_time2 = pd.Timestamp("2023-01-01 16:05:00")

df_t.loc[desired_time2:]

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-01 16:05:00,16563.20,16565.83,16562.00,16562.60,63.37417
2023-01-01 16:06:00,16563.08,16571.46,16562.43,16567.42,136.10204
2023-01-01 16:07:00,16567.88,16569.54,16565.87,16566.50,81.66886
2023-01-01 16:08:00,16566.50,16569.99,16565.43,16569.30,47.16452
2023-01-01 16:09:00,16569.08,16569.55,16566.07,16566.95,60.43350
...,...,...,...,...,...
2023-01-01 23:55:00,16609.48,16615.09,16609.19,16613.60,70.85806
2023-01-01 23:56:00,16613.20,16621.00,16612.15,16619.79,91.04320
2023-01-01 23:57:00,16620.20,16621.44,16617.19,16617.20,74.52966


In [65]:
position_arr = [False, False]
profits = []
# date = "2023-01-02"
# df_t = slicedf(date)

for date in opens.index:
    df_t = slicedf(date)
    if not any(position_arr):
        firstlevels = getlevels(date)
        secondlevels = getlevels(date, first=False)
    for index, row in df_t.iterrows():
        if not position_arr[0]:
            if row.Low <= firstlevels[0]:
                print('buy', index)
                position_arr[0] = True
                buy_1 = firstlevels[0]
        if position_arr[0] and not position_arr[1]:
            if row.Low <= secondlevels[0]:
                print("buy second", index)
                position_arr[1] = True
                buy_2 = secondlevels[0]
            if row.High >= firstlevels[1]:
                print('sell', index)
                position_arr[0] = False
                profits.append((firstlevels[1] - buy_1)/buy_1)
        if position_arr[1]:
            if row.High >= secondlevels[1]:
                print('sell second', index)
                position_arr[1] = False
                profits.append((secondlevels[1] - buy_2)/buy_2)